# 劣勾配法による最適化

* 参考: [Subgradient Method](https://web.stanford.edu/class/ee392o/subgrad_method.pdf)

まず劣微分についておさらいしておきましょう．
劣微分は微分不可能な関数に対しても拡張された微分の概念です．特に凸関数に対して劣微分が定義されます（凸じゃない関数でも考えることはできますが，あまり意味のある量になりません．）

凸関数$f$の点$x_0$における劣微分とは，任意の$x$に対して，次の条件を満たす$c$の集合です．

$$
f(x)-f\left(x_0\right) \geq c\left(x-x_0\right)
$$

劣微分は集合であることに注意しましょう．
この集合の要素を劣勾配といいます．


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera

x = np.linspace(-2.0, 2.0, 11)
y = np.abs(x)

fig = plt.figure()
camera = Camera(fig)


grad = np.linspace(-1.0, 1.0, 11)
for g in grad:
    plt.plot(x, y, color="blue")
    plt.scatter(x[5], y[5], s=30, color="red")
    plt.plot(x, g * x, color="green")
    camera.snap()

animation = camera.animate()
animation.save("./figs/subgradient.gif")
plt.close()

次のように，$f(x)=|x|$の$x=0$での劣微分は$[-1, 1]$の集合になります．
![subgrad](./figs/subgradient.gif)

## 劣勾配法

$f: \mathbb{R}^n\to \mathbb{R}$を凸関数とします．劣勾配法は次の更新を繰り返すことで$f$を最小化することを考えます．

$$x^{(k+1)}=x^{(k)} - \alpha_k g^{(k)}$$

ここで，$x^{(k)}$は$k$回目のイテレーションでの値，$g^{(k)}$は$x^{(k)}$での劣勾配，そして$\alpha_k$はステップサイズです．
劣勾配法は凸関数$f$が微分可能であれば，単純な勾配法と同じです．

劣勾配法では次のように過去の最小の値を保持しておくのが一般的です：

$$
f_{\text {best }}^{(k)}=\min \left\{f_{\text {best }}^{(k-1)}, f\left(x^{(k)}\right)\right\}
$$
このとき，
$$
f_{\text {best }}^{(k)}=\min \left\{f\left(x^{(1)}\right), \ldots, f\left(x^{(k)}\right)\right\}
$$
が成り立ちます．これは単調減少であることに注意しましょう．

$f^{\star}=\inf _x f(x)$を最適解とすると，次の結果が得られます．

* $\alpha_k$が定数の時：劣勾配法は$\lim _{k \rightarrow \infty} f_{\text {best }}^{(k)}-f^{\star}<\epsilon$を満たす解に収束する．（$f$が微分可能なときは，十分小さいステップサイズなら最適解に収束します．）
* $\sum_{k=1}^{\infty} \alpha_k^2<\infty, \quad \sum_{k=1}^{\infty} \alpha_k=\infty$を満たす時：$\lim _{k \rightarrow \infty} f\left(x^{(k)}\right)=f^{\star}$


### 収束の証明

$f$の最小値$x^*$が存在するとします．また，劣勾配が$\left\|g^{(k)}\right\|_2 \leq G$でバウンドされているとします．これは例えば$|f(u)-f(v)| \leq G\|u-v\|_2$のようなLipschitz条件で成立します．

早速劣勾配の証明を見ていきましょう．まず，次が成立します：

$$
\begin{aligned}
\left\|x^{(k+1)}-x^{\star}\right\|_2^2 & =\left\|x^{(k)}-\alpha_k g^{(k)}-x^{\star}\right\|_2^2 \\
& =\left\|x^{(k)}-x^{\star}\right\|_2^2-2 \alpha_k g^{(k) T}\left(x^{(k)}-x^{\star}\right)+\alpha_k^2\left\|g^{(k)}\right\|_2^2 \\
& \leq\left\|x^{(k)}-x^{\star}\right\|_2^2-2 \alpha_k\left(f\left(x^{(k)}\right)-f^{\star}\right)+\alpha_k^2\left\|g^{(k)}\right\|_2^2,
\end{aligned}
$$

最後の不等式では劣勾配の定義$f\left(x^{\star}\right) \geq f\left(x^{(k)}\right)+g^{(k) T}\left(x^{\star}-x^{(k)}\right)$を使っています．
この不等式を再帰的に適用すると，

$$
\left\|x^{(k+1)}-x^{\star}\right\|_2^2 \leq\left\|x^{(1)}-x^{\star}\right\|_2^2-2 \sum_{i=1}^k \alpha_i\left(f\left(x^{(i)}\right)-f^{\star}\right)+\sum_{i=1}^k \alpha_i^2\left\|g^{(i)}\right\|_2^2
$$

$\left\|x^{(k+1)}-x^{\star}\right\|_2^2 \geq 0$より，

$$
2 \sum_{i=1}^k \alpha_i\left(f\left(x^{(i)}\right)-f^{\star}\right) \leq\left\|x^{(1)}-x^{\star}\right\|_2^2+\sum_{i=1}^k \alpha_i^2\left\|g^{(i)}\right\|_2^2
$$

が成り立ちます．また，

$$
\sum_{i=1}^k \alpha_i\left(f\left(x^{(i)}\right)-f^{\star}\right) \geq\left(\sum_{i=1}^k \alpha_i\right) \min _{i=1, \ldots, k}\left(f\left(x^{(i)}\right)-f^{\star}\right)
$$

なので，二つを合体させると，

$$
f_{\text {best }}^{(k)}-f^{\star}=\min _{i=1, \ldots, k} f\left(x^{(i)}\right)-f^{\star} \leq \frac{\left\|x^{(1)}-x^{\star}\right\|_2^2+\sum_{i=1}^k \alpha_i^2\left\|g^{(i)}\right\|_2^2}{2 \sum_{i=1}^k \alpha_i}
$$

が成り立ちます．最後に仮定より劣勾配は$G$でバウンドされているので，

$$
f_{\text {best }}^{(k)}-f^{\star}=\min _{i=1, \ldots, k} f\left(x^{(i)}\right)-f^{\star} \leq \frac{\left\|x^{(1)}-x^{\star}\right\|_2^2+G^2 \sum_{i=1}^k \alpha_i^2}{2 \sum_{i=1}^k \alpha_i} .
$$

を得ます．$x^*$が$f$の任意の最小値であるとを考えると，$\operatorname{dist}\left(x^{(1)}, X^{\star}\right)$を$x^{(1)}$から最小値の集合までのユークリッド距離とすれば，

$$
f_{\text {best }}^{(k)}-f^{\star} \leq \frac{\operatorname{dist}\left(x^{(1)}, X^{\star}\right)^2+G^2 \sum_{i=1}^k \alpha_i^2}{2 \sum_{i=1}^k \alpha_i}
$$

と一般化することができます．

### $\alpha_k$が定数のとき

$\alpha_k=h$のときを考えましょう．このとき，

$$
f_{\text {best }}^{(k)}-f^{\star} \leq \frac{\operatorname{dist}\left(x^{(1)}, X^{\star}\right)^2+G^2 h^2 h}{2 h k}
$$

です．これより，右辺が$k \rightarrow \infty$で$G^2 h / 2$に収束することがわかります．

### $\|\alpha\|_2^2=\sum_{k=1}^{\infty} \alpha_k^2<\infty, \quad \sum_{k=1}^{\infty} \alpha_k=\infty$のとき

$$
f_{\text {best }}^{(k)}-f^{\star} \leq \frac{\operatorname{dist}\left(x^{(1)}, X^{\star}\right)^2+G^2\|\alpha\|_2^2}{2 \sum_{i=1}^k \alpha_i}
$$

なので，右辺は$0$に収束します．

## 射影劣勾配法

射影劣勾配法では次の問題を扱います．

$$
\begin{array}{ll}
\operatorname{minimize} & f(x) \\
\text { subject to } & x \in \mathcal{C}
\end{array}
$$

ここで，$C$は凸集合です．射影劣勾配法は次の更新を行います：

$$
x^{(k+1)}=P\left(x^{(k)}-\alpha_k g^{(k)}\right)
$$

ここで，$P$は$C$に対する（ユークリッド）射影です．
これも通常の劣勾配法のような収束が保証できます．

$z^{(k+1)}=x^{(k)}-\alpha_k g^{(k)}$とします．劣勾配法と同じ証明をして，

$$
\begin{aligned}
\left\|z^{(k+1)}-x^{\star}\right\|_2^2 & =\left\|x^{(k)}-\alpha_k g^{(k)}-x^{\star}\right\|_2^2 \\
& =\left\|x^{(k)}-x^{\star}\right\|_2^2-2 \alpha_k g^{(k) T}\left(x^{(k)}-x^{\star}\right)+\alpha_k^2\left\|g^{(k)}\right\|_2^2 \\
& \leq\left\|x^{(k)}-x^{\star}\right\|_2^2-2 \alpha_k\left(f\left(x^{(k)}\right)-f^{\star}\right)+\alpha_k^2\left\|g^{(k)}\right\|_2^2 .
\end{aligned}
$$

が成り立ちます．ここで，

$$
\left\|x^{(k+1)}-x^{\star}\right\|_2=\left\|P\left(z^{(k+1)}\right)-x^{\star}\right\|_2 \leq\left\|z^{(k+1)}-x^{\star}\right\|_2,
$$

であることに注意しましょう．つまり，$P$によって$C$に射影されると，$C$のどの点に対しても以前よりも近くなるので，最適値にも近づきます．
よって，

$$
\left\|x^{(k+1)}-x^{\star}\right\|_2^2 \leq\left\|x^{(k)}-x^{\star}\right\|_2^2-2 \alpha_k\left(f\left(x^{(k)}\right)-f^{\star}\right)+\alpha_k^2\left\|g^{(k)}\right\|_2^2,
$$

が成り立ちます．後は通常の劣勾配法と同じです．